# Intro

This notebook aims at downloading sequences infos(images and bboxes) from a Distant/Real Pyronear Alert API isntance in order to recreate alerts/sequences with ease in local dev environnement

You need to have accesses to a distant api and provide ```DISTANT_API_URL```, ```DISTANT_ALERT_API_LOGIN```, ```DISTANT_ALERT_API_PASSWORD```

And obvisouly, fill ```SEQUENCE_ID_LIST``` with the sequence id you want in order to download related data


If you do not have anything from above, do not worry, you can use the notebook "send_real_alerts" to send some alerts to local env dev

In [1]:
BASE_DIRECTORY = "alerts" # directory where to put sequences data

In [2]:
import requests
from dotenv import load_dotenv
import os

from pyroclient import Client

load_dotenv("../.env")
DISTANT_API_URL = os.environ.get("DISTANT_API_URL")
DISTANT_ALERT_API_LOGIN = os.environ.get("DISTANT_ALERT_API_LOGIN")
DISTANT_ALERT_API_PASSWORD = os.environ.get("DISTANT_ALERT_API_PASSWORD")

In [3]:
token = requests.post(
    f"{DISTANT_API_URL}api/v1/login/creds",
    data={"username": DISTANT_ALERT_API_LOGIN, "password": DISTANT_ALERT_API_PASSWORD},
    timeout=5,
).json()['access_token']

api_client = Client(token, DISTANT_API_URL)

# get cameras -> in order to get cam name from cam id
cameras = api_client.fetch_cameras().json()

# define base directory
base_dir = os.path.join(BASE_DIRECTORY)
os.makedirs(base_dir, exist_ok=True)


In [ ]:
response = api_client.fetch_sequences_from_date("2025-08-26", limit=50)
api_sequences = pd.DataFrame(response.json())
api_sequences.head()

In [ ]:
SEQUENCE_ID_LIST = [] #fullfill this with alertes id

In [8]:
# you can also reproduce all alerts from this day
if len(SEQUENCE_ID_LIST)==0:
    SEQUENCE_ID_LIST = list(api_sequences["id"])

In [ ]:
def dl_seqs_in_target_dir(sequence_id_list, target_dir, api_client):
    """
    Download sequences from sequence_id_list to target_dir, using an instanciated api_client (of distant Pyronear alert API)

    """
    
    for seq_id in sequence_id_list: 
        sequences = api_client.fetch_sequences_detections(sequence_id=seq_id, limit=10, desc=False).json()
    
        cam_name = [item['name'] for item in cameras if item['id'] == sequences[0]['camera_id']][0]
        created_at_rounded = sequences[0]["created_at"].split('.')[0].replace(':', '-').replace('T', '_')
        cam_id_distant_api = sequences[0]["camera_id"]
        print(f"== Download Alerts data for sequence ID {seq_id} - camera {cam_name} at {created_at_rounded}")
        
        alert_dir = os.path.join(f"{cam_id_distant_api}_{cam_name}_{created_at_rounded}")
        image_dir = os.path.join(target_dir,alert_dir,  "images")
        pred_dir = os.path.join(target_dir, alert_dir, "labels_predictions")
        os.makedirs(image_dir, exist_ok=True)
        os.makedirs(pred_dir, exist_ok=True)
    
        for seq in sequences:
            
            # bbox
            #yolo_format_bbox = ' '.join(map(str,ast.literal_eval(seq["bboxes"])[0]))
            bboxes = seq["bboxes"]
    
            bbox_file_name = seq["bucket_key"][:-4] + ".txt"
            
            with open(os.path.join(target_dir, alert_dir, "labels_predictions",bbox_file_name), 'w') as f:
                f.write(bboxes)
    
            url = seq['url']
            nom_fichier = seq["bucket_key"]
            # image
            response = requests.get(url, stream=True)
            if response.status_code == 200:
                full_img_path = os.path.join(image_dir, nom_fichier)
                with open(full_img_path, 'wb') as f:
                    for chunk in response.iter_content(1024):
                        f.write(chunk)
            else:
                print(f"Error during download.")
    print("Download complete")
    

# Download sequences from SEQUENCE_ID_LIST list

In [5]:
single_alerts = os.path.join(BASE_DIRECTORY, "single_sequences")
os.makedirs(single_alerts, exist_ok=True)

dl_seqs_in_target_dir(SEQUENCE_ID_LIST, single_alerts, api_client)

== Download Alerts data for sequence ID 15462 - camera serre-de-barre-01 at 2025-09-29_13-56-58
== Download Alerts data for sequence ID 15463 - camera brison-03 at 2025-09-29_13-58-36
== Download Alerts data for sequence ID 15561 - camera pouncho-agast-01 at 2025-10-01_06-18-57
== Download Alerts data for sequence ID 15526 - camera pouncho-agast-01 at 2025-09-30_15-29-49
== Download Alerts data for sequence ID 13880 - camera fontaneilles-01 at 2025-09-04_07-34-32
== Download Alerts data for sequence ID 13572 - camera pouncho-agast-01 at 2025-08-29_06-59-39
== Download Alerts data for sequence ID 13537 - camera pouncho-agast-01 at 2025-08-28_15-31-17
== Download Alerts data for sequence ID 12997 - camera pouncho-agast-01 at 2025-08-17_15-52-27
== Download Alerts data for sequence ID 12562 - camera pouncho-agast-01 at 2025-08-06_12-32-09
== Download Alerts data for sequence ID 15376 - camera pouncho-agast-02 at 2025-09-28_10-52-39
== Download Alerts data for sequence ID 15526 - camera po

## Download triangulated alerts


In [6]:

TRIANGULATED_SEQUENCE_LIST = [10535, 10537, 10538, 15462, 15463]
TRIANGULATED_DIRECTORY= "triangulated_sequences"
triangulated_dir = os.path.join(BASE_DIRECTORY, TRIANGULATED_DIRECTORY)
os.makedirs(triangulated_dir, exist_ok=True)

dl_seqs_in_target_dir(TRIANGULATED_SEQUENCE_LIST, triangulated_dir, api_client)

== Download Alerts data for sequence ID 10535 - camera moret-sur-loing-01 at 2025-07-14_11-22-05
== Download Alerts data for sequence ID 10537 - camera nemours-02 at 2025-07-14_11-24-10
== Download Alerts data for sequence ID 10538 - camera croix-augas-02 at 2025-07-14_11-24-31
== Download Alerts data for sequence ID 15462 - camera serre-de-barre-01 at 2025-09-29_13-56-58
== Download Alerts data for sequence ID 15463 - camera brison-03 at 2025-09-29_13-58-36
Download complete
